<p align="center">
    <a
    href="https://youtu.be/dcb4Ckpkx2o"
    target="_blank"
    rel="noopener noreferrer">
        <img
        alt="Night Sky Latent Walk"
        width="350" height="350"
        src="https://github.com/ArthurFDLR/GANightSky/blob/main/.github/random_walk.gif?raw=true">
    </a>
</p>

# 🚀 StyleGan2-ADA for Google Colab



1.   [Install StyleGAN2-ADA on your Google Drive](#scrollTo=5YcUMPQp6ipP)
2.   [Train a custom model](#scrollTo=Ti11YiPAiQpb)
3.   [Generate images from pre-trained model](#scrollTo=f0A9ZNtferpk)
4.   [Latent space exploration](#scrollTo=5yG1UyHXXqsO)


## Install StyleGAN2-ADA on your Google Drive

StyleGAN2-ADA only works with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.


In [ ]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Thu Nov 25 16:06:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Then, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


Finally, run this cell to install StyleGAN2-ADA on your Drive. If you’ve already installed the repository, it will skip the installation process and only check for updates. If you haven’t installed it, it will install all the necessary files. Beside, **in**, **out**, **datasets** and **training** folders are generated for data storage. Everything will be available on your Google Drive in the folder **StyleGAN2-ADA** even after closing this Notebook.

In [ ]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "ArthurFDLR"
!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


## Train a custom model

Once you have installed StyleGAN2-ADA on your Google Drive and set up the working directory, you can upload your training dataset images in the associated folder.

In [ ]:
dataset_name = 'NightSky'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
if datasets_source_path.is_dir():
    print("Dataset ready for import.")
else:
    print('Upload your images dataset as {}'.format(datasets_source_path))

Upload your images dataset as /content/drive/MyDrive/StyleGAN2-ADA/datasets/source/NightSky.zip


Unfortunately, large datasets might exceed the Google Drive quota after a few training batches. Indeed, StyleGAN2 download datasets multiple times during training. You might have to import your dataset in the local storage session. However, large files cannot be copy/paste from Drive *(Input/Output error)*. 

Run this cell to download your zipped dataset from your Drive and unzip it in the local session.

In [ ]:
#local_dataset_path = content_path / 'dataset'
#if not local_dataset_path.is_dir():
#   print("Importing dataset...")
#   %mkdir "{local_dataset_path}"
#   %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
#   print("Zip file succesfuly imported")
#else:
#   print('Zip file allready imported')
#import zipfile
#with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
#    zip_ref.extractall(str(local_dataset_path))
#print('Extraction completed')

### Convert dataset to .tfrecords

Next, we need to convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.

This can take a while.

In [ ]:
local_images_path ="/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/NightSky"

In [ ]:
local_dataset_path = "/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/tfr/"

In [ ]:
!python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
"{local_dataset_path}" "{local_images_path}"

Loading images from "/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/NightSky"
Creating dataset "/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/tfr/"
/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 387 images.


There are numerous arguments to tune the training of your model. To obtain nice results, you will certainly have to experiment. Here are the most popular parameters:


*   *mirror:* Should the images be mirrored vertically?
*   *mirrory:* Should the images be mirrored horizontally?
*   *snap:* How often should the model generate image samples and a network pickle (.pkl file)?
*   *resume:* Network pickle to resume training from?

To see all the options, run the following ```help``` cell.

Please note that Google Colab Pro gives access to V100 GPUs, which drastically decreases (~3x) processing time over P100 GPUs.

In [ ]:
!python "{stylegan2_repo_path / 'train.py'}" --help

In [ ]:
training_path = project_path / 'training' / dataset_name
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 10
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'geom'

#resume_from = 'brecahad512'
resume_from = 'training/pickles/FreaGAN.pkl'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" --resume="{resume_from}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} #--dry-run

tcmalloc: large alloc 4294967296 bytes == 0x557c7dfda000 @  0x7f37df4f8001 0x7f37dc6fb54f 0x7f37dc74bb58 0x7f37dc74fb17 0x7f37dc7ee203 0x557c7718c544 0x557c7718c240 0x557c77200627 0x557c771faced 0x557c7718e48c 0x557c771cf159 0x557c771cc0a4 0x557c7718cd49 0x557c7720094f 0x557c771fa9ee 0x557c770cce2b 0x557c771fcfe4 0x557c771fa9ee 0x557c770cce2b 0x557c771fcfe4 0x557c771faced 0x557c770cce2b 0x557c771fcfe4 0x557c7718dafa 0x557c771fb915 0x557c771fa9ee 0x557c771fa6f3 0x557c772c44c2 0x557c772c483d 0x557c772c46e6 0x557c7729c163
tcmalloc: large alloc 4294967296 bytes == 0x557d7dfda000 @  0x7f37df4f61e7 0x7f37dc6fb46e 0x7f37dc74bc7b 0x7f37dc74c35f 0x7f37dc7ee103 0x557c7718c544 0x557c7718c240 0x557c77200627 0x557c771fa9ee 0x557c7718dbda 0x557c771fc737 0x557c771fa9ee 0x557c7718dbda 0x557c771fc737 0x557c771fa9ee 0x557c7718dbda 0x557c771fc737 0x557c7718dafa 0x557c771fb915 0x557c771fa9ee 0x557c7718dbda 0x557c771ffd00 0x557c771fa9ee 0x557c7718dbda 0x557c771fc737 0x557c771faced 0x557c7718e48c 0x557c771c

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6

generation_from = 'training/NightSky/00024-tfr-mirror-auto1-geom-resumecustom/network-snapshot-000000.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}